In [1]:
from ansys.mapdl.core import launch_mapdl

def launch_mapdl_on_available_port(starting_port=50052, max_attempts=10):
    for i in range(max_attempts):
        port = starting_port + i
        try:
            mapdl = launch_mapdl(port=port)
            print(f"MAPDL launched successfully on port {port}")
            return mapdl
        except Exception as e:
            print(f"Failed to launch MAPDL on port {port}: {e}")
    raise RuntimeError("Could not launch MAPDL on any available port")

# Use a função para iniciar uma instância do MAPDL
mapdl = launch_mapdl_on_available_port()

MAPDL launched successfully on port 50052


In [2]:
# Limpar qualquer dado existente
mapdl.clear()

# Entrar no pré-processador
mapdl.prep7()

# Definir o tipo de elemento (BEAM188) e suas propriedades
mapdl.et(1, "BEAM188")

# Propriedades do material
mapdl.mp("EX", 1, 2e7)  # Módulo de elasticidade em N/cm2
mapdl.mp("PRXY", 1, 0.27)  # Coeficiente de Poisson

# Propriedades da seção da viga em centímetros
sec_num = 1
mapdl.sectype(1, "BEAM", "CSOLID")
mapdl.secoffset("CENT")
mapdl.secdata(0.5)

# Definir os pontos iniciais e finais das estacas
pontos_iniciais = [
    [0, 3, 0],
    [2, 3, 0],
    [4, 3, 0],
    [6, 3, 0],
    [3, 0.0, 0],
    [4, 0.0, 0],
    [0, -3, 0],
    [2, -3, 0],
    [4, -3, 0],
    [6, -3, 0]
]

pontos_finais = [
    [0, 3, -20],
    [2, 3, -20],
    [4, 3, -20],
    [6, 3, -20],
    [3, 0.0, -20],
    [4, 0.0, -20],
    [0, -3, -20],
    [2, -3, -20],
    [4, -3, -20],
    [6, -3, -20]
]

# Definir um deslocamento para evitar conflito com os keypoints criados automaticamente
offset_keypoints = 0

# Adicionar os keypoints iniciais e finais
for i, (p_inicial, p_final) in enumerate(zip(pontos_iniciais, pontos_finais), start=1):
    mapdl.k(i + offset_keypoints, *p_inicial)
    mapdl.k(i + len(pontos_finais) + offset_keypoints, *p_final)
    # Criar a linha (elemento de viga) entre os keypoints iniciais e finais
    mapdl.l(i + offset_keypoints, i + len(pontos_finais) + offset_keypoints)

# Malhar as linhas
mapdl.type(1)
mapdl.esize(1)
mapdl.lesize("ALL")
mapdl.lmesh("ALL")

# Definição e malha da casca
mapdl.et(2, "SHELL181")
mapdl.mp("EX", 2, 2e5)  # Módulo de elasticidade
mapdl.mp("PRXY", 2, 0.3)  # Coeficiente de Poisson
mapdl.rectng(0, 6, -3, 3)  # Cria um retângulo
mapdl.sectype(2, "SHELL")  # Define o tipo de seção
mapdl.secdata(0.1)  # Define a espessura da seção
mapdl.esize(0.1)  # A densidade de elementos
mapdl.amesh("all")

# Contagem inicial de nós
initial_node_count = mapdl.get_value(entity="NODE", entnum=0, item1="COUNT")
print(f"Initial node count: {initial_node_count}")

# Merge all
mapdl.nsel("s", "loc", "z", 0, 1e5)
mapdl.nummrg("NODE", 1e-5)
mapdl.nsel("ALL")

# Contagem final de nós
final_node_count = mapdl.get_value(entity="NODE", entnum=0, item1="COUNT")
print(f"Final node count: {final_node_count}")

# Verificação do resultado da mesclagem
if initial_node_count > final_node_count:
    print("Node merging was successful.")
else:
    print("Node merging did not reduce the node count.")

# Visualizar a malha
mapdl.eplot()

mapdl.allsel()
mapdl.solution()

# Definir as condições de contorno
mapdl.nsel("s", "loc", "z", -20.001, -19.999)
mapdl.d("all", "all")

# Plot das condições de contorno
mapdl.allsel()
mapdl.eplot(plot_bc=True, show_node_numbering=False)

# Definir a gravidade
mapdl.acel(0, 0, 9.81)

# Resolver o problema
mapdl.antype("static")
mapdl.solve()
print(mapdl.solve())
mapdl.finish()
mapdl.exit()


Initial node count: 3931.0
Final node count: 3921.0
Node merging was successful.


Widget(value='<iframe src="http://localhost:57686/index.html?ui=P_0x1d7746ace50_0&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:57686/index.html?ui=P_0x1d700133490_1&reconnect=auto" class="pyvis…

MapdlExitedError: MAPDL server connection terminated

In [ ]:
from ansys.mapdl.core import launch_mapdl
mapdl.finish()
mapdl.exit()

MapdlExitedError: MAPDL server connection terminated